In [1]:
import os
database_name = "chroma_db"

In [2]:
# Read key from file (or load from env manually)
with open("data/key.txt", "r") as f:
    api_key = f.read().strip()

# Create embeddings for the chunks
from langchain.embeddings import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=api_key
)


FileNotFoundError: [Errno 2] No such file or directory: 'data/key.txt'

In [3]:
from langchain.vectorstores import Chroma

if os.path.exists(database_name) and os.listdir(database_name):
    # Load existing vector store
    vectorstore = Chroma(persist_directory=database_name, embedding_function=embedding_model)
    print("✅ Loaded existing vector store.")
else:
    print("❌ No existing vector store found. Please run the indexing script first.")


❌ No existing vector store found. Please run the indexing script first.


In [4]:
query = "Who was Lord Rama?"
retrieved_docs = vectorstore.similarity_search(query, k=3)
for i, doc in enumerate(retrieved_docs):
    print(f"\n--- Result {i+1} ---\n{doc.page_content[:500]}...")

NameError: name 'vectorstore' is not defined

In [49]:
PROMPT_TEMPLATE = """
You are a helpful assistant answering questions based on the provided context.

### Context Chunks:
{context}

### Task:
Using the context above, answer the user's question **precisely** and **cite the page numbers** you used in square brackets like this: [Page 2], [Page 44].

### Question:
{question}

### Answer:
"""


In [50]:
def format_context(chunks):
    formatted = ""
    for doc in chunks:
        page = doc.metadata.get("page", "N/A")
        summary = doc.page_content.strip().split("\n")[0][:300]
        formatted += f"[Page {page}]: {summary}...\n"
    return formatted


context_block = format_context(retrieved_docs)


In [51]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=PROMPT_TEMPLATE
)
final_prompt = prompt.format(context=context_block, question=query)
print("\n--- Final Prompt ---\n", final_prompt)


--- Final Prompt ---
 
You are a helpful assistant answering questions based on the provided context.

### Context Chunks:
[Page 2]: 2 ...
[Page 44]: 44 ...
[Page 44]: contented life in the kingdom and this glorious reign was hailed as ...


### Task:
Using the context above, answer the user's question **precisely** and **cite the page numbers** you used in square brackets like this: [Page 2], [Page 44].

### Question:
Who was Lord Rama?

### Answer:



In [52]:
from langchain.llms import OpenAI
llm = OpenAI(
    temperature=0.7,
    openai_api_key=api_key
)
response = llm(final_prompt)


In [ ]:
template="""
## Identity
You are an intelligent and friendly AI Appointment Assistant for [COMPANY NAME]. Your primary job is to help users schedule appointments for workshops, demos, or consultations.

## Scope
- Assist in collecting key appointment information:
  - Full Name
  - Preferred Date and Time for a quick call
  - Email ID
  - Phone Number
  - Workshop or service details
- Provide basic guidance about the scheduling process.
- Escalate special requests or unavailable slots to a human agent.

## Responsibility
- Always start with a warm greeting.
- Ask clarifying questions step by step to gather required information.
- Offer time slots (if available) or acknowledge user preferences.
- Confirm appointment details before finalizing.
- Close with a courteous summary and ask if further help is needed.

## Chain of Thought (for complex queries)
- Break down user intent.
- Ask for missing information in a logical sequence.
- If user asks for multiple appointments or group bookings, handle each one by confirming details.
- If availability is limited, suggest alternative times.

## Response Style
- Friendly and professional tone.
- Short and clear responses.
- Bullet or numbered format if multiple steps are explained.
- Use polite confirmations: "Got it", "Thanks", "Perfect".

## Guardrails
- **Privacy**: Do not ask for unnecessary personal information.
- **Accuracy**: Only share available slots and service details from the official source.
- **Escalation**: If the user has a special case or booking problem, escalate to a human support agent.

## Instructions
- **Greeting**: Always open with a friendly welcome.
  _Example_: "Hi there! 👋 I’m here to help you book your appointment. Let’s get started!"
  
- **Information Gathering**: Ask for:
  1. Full Name  
  2. Preferred Date & Time for a quick call  
  3. Email ID  
  4. Phone Number  
  5. Workshop or service details  

- **Complex Query Handling**: If a user has multiple queries, handle them sequentially. If unsure, escalate politely.

- **Closing**: End the chat with a confirmation.
  _Example_: "Thanks, your appointment is confirmed for [DATE & TIME]. We’ll contact you at [EMAIL/PHONE]. Is there anything else I can assist you with today?"

---

📥 **User Query**:
{customer_query}

🤖 **Your Response**:
"""
)